In [ ]:
pip install git+https://github.com/LucianoPereiraValenzuela/ECC_2025_testing.git

In [17]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_ibm_runtime.fake_provider import FakeBurlingtonV2 as FakeDevice
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from ECC2025.testing import test_2

En el desafio anterior vimos la forma más efficiente de implementar un estado GHZ en 4 qubits. Sin embargo, esta estrategia puede no ser siempre la más inteligente. Esto se debe a que los computadores cuánticos tienes conectividad limitada, es decir, __no todos los qubits estan conectados entre si__. Tomemos por ejemplo el siguiente dispositivo cuántico de IBM de 5 qubits:

![texto del enlace](https://github.com/LucianoPereiraValenzuela/ECC_2025_testing/blob/589e681af9966dc4f9c7a2d97a54737389931df6/img/burlington.jpg?raw=true)

Podemos ver que el qubit 0 esta solamente conectado con el qubit 1, mientras que el qubit 1 esta conectado con el qubit 0, 2 y 3.

Esto tiene un gran impacto en la calidad de ejecución de los circuitos, pues nos vemos obligados a descomponer operaciones entre qubits no vecinos en una cadena de puertas entre qubits conectados. Tomemos por ejemplo el circuito del estado GHZ:

In [ ]:
n_qubits = 4
qc_ghz = QuantumCircuit( n_qubits )
qc_ghz.h(0)
for j in range(n_qubits-1):
    qc_ghz.cx(j,j+1)

qc_ghz.draw('mpl')

Si queremos implementar este estado entre los qubits 0, 1, 2, y 3 del dispositivo anterior, vemos que los qubits 2 y 3 no estan conectados, por lo que debemos descomponer la última CNOT en una secuencia de 4 CNOTs:

In [ ]:
qc_ghz_device = QuantumCircuit( n_qubits )
qc_ghz_device.h(0)
for j in range(n_qubits-2):
    qc_ghz_device.cx(j,j+1)

qc_ghz_device.cx(2,1)
qc_ghz_device.cx(1,2)
qc_ghz_device.cx(2,1)
qc_ghz_device.cx(1,3)

qc_ghz_device.draw('mpl')

Podemos verificar que este circuito implementa correctamente el estado GHZ.

In [ ]:
state = Statevector( qc_ghz_device )
state.draw("latex")

¡Sin embargo, la profundidad del circuito aumentó!

In [ ]:
qc_ghz_device.depth()

Este aumento de profundidad se traduce en que la ejecución de este circuito en un computador cuántico real será de peor calidad, pues hay más puertas cuánticas involucradas. Veamoslo en una simulación considerando un modelo realista del dispositivo.

In [ ]:
device_backend = FakeDevice()
simulator_noise = AerSimulator.from_backend(device_backend)
qc_ghz_device_measured = qc_ghz_device.copy()
qc_ghz_device_measured.measure_all()
qc_ghz_device_measured =  transpile( qc_ghz_device_measured, device_backend, optimization_level=0 )
counts_device = simulator_noise.run( qc_ghz_device_measured ).result().get_counts()

counts_ideal = { '0000':500, '1111':500  }

plot_histogram( [counts_ideal, counts_device], legend=['Ideal','Ruidoso'] )

__Desafio:__ Construya un circuito cuántico que implemente un estado GHZ de 4 qubits en el dispisitivo cuántico anterior que tenga una profundidad de 4. Escriba su solución en el circuito ``qc_ghz_op``, definido en la siguiente celda.

In [ ]:
qc_ghz_op = QuantumCircuit( n_qubits )

### escribe tu solución aca ###

###############################

qc_ghz_op.draw('mpl')

Podemos verificar si el circuit funciona mejor que el anterior a través de una simulación con ruido. Como la simulación es un experimento aleatorio, quizás debas ejecutarlo varias veces para convencerte del resultado.

In [ ]:
qc_ghz_op_measured = qc_ghz_op.copy()
qc_ghz_op_measured.measure_all()
qc_ghz_op_measured =  transpile( qc_ghz_op_measured, device_backend, optimization_level=0 )
counts_op = simulator_noise.run( qc_ghz_op_measured ).result().get_counts()

plot_histogram( [counts_ideal, counts_device, counts_op], legend=['Ideal','Ruidoso', 'Optimizado']  )

In [ ]:
## Esto verifica tu solución, no cambiar!!!! ##
test_2( qc_ghz_op )